
# Geração de malha não-estruturada e não-convexa

O código a seguir realiza a criação de uma malha pelo método incremental e randômico.

In [ ]:
%matplotlib notebook

In [ ]:
from collections import deque   
import numpy as np
import numpy.ma as ma           # mask arrays
import matplotlib.pyplot as plt
     
    
class TriangleElement:
    def __init__(self, A, B, C):
        self.vertices = [A, B, C]  # vertices sentido anti-horario
        self.child = ["", "", ""]  # chaves dos triangulos filhos
        self.live = True           # verdadeiro se triangulo esta ativo
        
class Delaunay:
    def __init__(self, points):
        self.points = points # pontos a serem trinagulados
        self.triangles = {}  # armazena as instancias dos triangulos 
        self.nodes = {}      # armazena as ligacoes entre nos (i.e. grafo)
        self.fuzz = 1.0E-6   # deslocamento para pontos colineares
             
    def generate_triangulation(self):
        """
        Realiza a Triangulacao de Delaunay.
        """
        bigscale = 100.0
        delx, dely = np.ptp(self.points, axis=0) # peak to peak
        xl, yl = self.points.min(axis=0) # menores coordenadas
        xh, yh = self.points.max(axis=0) # menores coordenadas
        points = [np.array([0.5*(xl+xh), yh+bigscale*dely]),
                  np.array([xl-0.5*bigscale*delx, yl-0.5*bigscale*dely]),
                  np.array([xh+0.5*bigscale*delx, yl-0.5*bigscale*dely])]
        self.points = np.concatenate((np.array(points), self.points))
        self.add_triangle(0, 1, 2) # indice do triangulo maior
        # nao sorteia pontos do triangulo maior
        for i in np.random.permutation(np.arange(3, len(self.points))):
            self.insert_point(i)
        output = []
        for triangle in self.triangles.values():
            if triangle.live: 
                # se esta ligado ao triangulo grande
                if any([vertice < 3 for vertice in triangle.vertices]): 
                    triangle.live = False
                else: 
                    # renumera os vertices. ignora os indices triangulo grande
                    output.append([vertex-3 for vertex in triangle.vertices]) 
        return (self.points[3:], np.array(output))
    
    def insert_point(self, P):
        """
        Insere um ponto P na triangulacao.
        """
        stack = deque() 
        key = self.which_cointains_pt(P)
        A, B, C = self.triangles[key].vertices 
        tri_0 = self.add_triangle(P, A, B)
        stack.append([P, A, B])
        tri_1 = self.add_triangle(P, B, C)
        stack.append([P, B, C])
        tri_2 = self.add_triangle(P, C, A)
        stack.append([P, C, A])
        self.define_child(A, B, C, tri_0, tri_1, tri_2)
        while len(stack):
            D, E, F = stack.pop() 
            G = self.fourth_point(E, F)
            if G: # se existe o quarto ponto
                if self.in_circle(D, E, F, G): # precisa dar um flip
                    # cria os dois novos triangulos
                    tri_0 = self.add_triangle(D, G, F)
                    tri_1 = self.add_triangle(D, E, G)
                    # reorganiza a paternidade
                    self.define_child(D, E, F, tri_0, tri_1, "")
                    self.define_child(G, F, E, tri_0, tri_1, "")
                    # remove arestas
                    self.nodes.pop(hash("{}-{}".format(E, F)))
                    self.nodes.pop(hash("{}-{}".format(F, E)))
                    # empilha os novos triangulos para legalizar
                    stack.append((D, G, F))
                    stack.append((D, E, G))
            
    def which_cointains_pt(self, P): 
        """
        Retorna o valor da chave do triangulo que contem o ponto P.
        A pesquisa eh feita explorando em profundidade a arvore de triangulos 
        ate encontrar o triangulo vivo que contem o ponto P em questao. 
        NOTA: Todos os pontos estao dentro do grande triangulo inicial, 
              portanto um ponto sempre estara dentro de algum triangulo.
              Apesar disso, o ponto pode estar na fronteira de dois triangulos
              nesse caso a funcao reposiciona o ponto e tenta novamente.
        """
        key = self.triangle_key(0, 1, 2)    # chave do triangulo raiz
        while not self.triangles[key].live: # itera ate triangulo vivo
            found = False
            while True: # loop garante pontos nao alinhados
                for i in range(3):
                    next_key = self.triangles[key].child[i]
                    if next_key: # se a proxima eh valida (i.e. nao "")
                        if self.contains(self.triangles[next_key], P):
                            found = True
                            break
                if found:
                    break
                else: # reposiciona o ponto
                    self.points[P][0] += np.random.choice([-1, 1]) * self.fuzz
                    self.points[P][1] += np.random.choice([-1, 1]) * self.fuzz
            key = next_key # avanca na estrutura da arvore
        return key
    
    def add_triangle(self, A, B, C):
        """
        Armazena os novos triangulos gerados e armazena as novas ligacoes entre
        os pontos A, B, C.
        Os pontos A, B, C sao passados na ordem anti-horaria!
        """
        self.nodes[hash("{}-{}".format(B, C))] = A
        self.nodes[hash("{}-{}".format(C, A))] = B
        self.nodes[hash("{}-{}".format(A, B))] = C
        key = self.triangle_key(A, B, C)
        self.triangles[key] = TriangleElement(A, B, C)
        return key
    
    def define_child(self, A, B, C, tri_0, tri_1, tri_2):
        """
        Define os novos triangulos filhos do triangulo ABC.
        Torna o triangulo pai inativo.
        """
        key = self.triangle_key(A, B, C)
        self.triangles[key].child = [tri_0, tri_1, tri_2] 
        self.triangles[key].live = False
        
    def in_circle(self, A, B, C, D):
        """ 
        Verifica se D esta dentro, fora ou sobre a circunferencia que contem
        os pontos A, B, C.
        https://en.wikipedia.org/wiki/Circumscribed_circle
        """
        #TODO: testar se nao eh mais rapido o determinante com o numpy e a 
        #      equacao na pagina da wiki delaunay
        A = self.points[A]
        A_x, A_y = A
        B_x, B_y = self.points[B]
        C_x, C_y = self.points[C]
        D = self.points[D]
        d = 2*(A_x*(B_y-C_y) + B_x*(C_y-A_y) + C_x*(A_y-B_y))
        norm_A = A_x**2 + A_y**2
        norm_B = B_x**2 + B_y**2
        norm_C = C_x**2 + C_y**2
        U = np.array([norm_A*(B_y-C_y) + norm_B*(C_y-A_y) + norm_C*(A_y-B_y),
            -(norm_A*(B_x-C_x) + norm_B*(C_x-A_x) + norm_C*(A_x-B_x))]) / d
        return (np.linalg.norm(U - D) - np.linalg.norm(U - A)) < 0

    def contains(self, triangle, P):
        """ 
        Verifica se pertence o ponto esta contido no triangulo 
        https://math.stackexchange.com/questions/51326/determining-if-an-
        arbitrary-point-lies-inside-a-triangle-defined-by-three-points
        """
        A, B, C = [self.points[i] for i in triangle.vertices]
        P = self.points[P]
        cross = [np.sign(np.cross(B-A, P-A)),
                 np.sign(np.cross(C-B, P-B)),
                 np.sign(np.cross(A-C, P-C))]
        if 0.0 in cross: 
            return False
        return cross[0] == cross[1] and cross[1] == cross[2]
    
    def triangle_key(self, A, B, C):
        """
        Dados tres pontos A, B, C cria uma chave unica como uma string.
        NOTA: ordenacao dos pontos garante chave unica independen da ordem ABC.
        """
        return "-".join(map(str, sorted([A, B, C])))
    
    def fourth_point(self, A, B):
        """
        Pega o quarto ponto na direcao da aresta AB.
        Caminha no sentido horario para pegar o quarto ponto.
        """
        edge_key = hash("{}-{}".format(B, A))
        return self.nodes[edge_key] if edge_key in self.nodes else None 

    def plot(self, plot_node_num=False):
        """
        Plota o grafico.
        plot_node_num: plota de cada no.
        """
        points, triangles = self.generate_triangulation()
        plt.triplot(points[:, 0], points[:, 1], triangles, "go-")
        if plot_node_num: # plota o numero dos nos
            for j, p in enumerate(self.points):
                plt.text(p[0]-0.03, p[1]+0.03, j, ha='right')
        plt.show()


# Encontra o contorno


Encontra o contorno do objeto. A função encontra dois contornos fechados que são plotados em cores distintas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import imageio

# carrega imagem e aplica limiar de 20 (cinza) para binarizar 
img = imageio.imread("gear_2.png", pilmode="L")
bin_img = np.where(img < 20, 0, 1) #limiar =20 #0 se for preto, 1 se for branco

# encontra os contornos
contours = measure.find_contours(bin_img, 0.8)

# plota a imagem
fig, ax = plt.subplots()
ax.imshow(bin_img, cmap=plt.cm.gray, alpha=0.35)

# plota o contorno
#for n, contour in enumerate(contours):
#    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis("image")
plt.show()

In [ ]:
#from matplotlib import path
mask_value = 0 #valor binario do objeto
num_pts = 15
size_x, size_y = img.shape 
step_x = size_x/num_pts
step_y = size_y/num_pts
pts_x = np.arange(step_x, size_x, step_x)
pts_y = np.arange(step_y, size_y, step_y)
xx, yy = np.meshgrid(pts_x, pts_y)
s_x, s_y  = xx.ravel(), yy.ravel()

mask = bin_img[s_x, s_y] == mask_value #bin_img[s_x, s_y] == mask_value 
x = s_x[mask]
y = s_y[mask]

# distorce a malha
g = .8 # fator de distorcao
x += np.random.randint(-g*(0.5*size_x/num_pts), g*(0.5*size_x/num_pts), size=x.shape)
y += np.random.randint(-g*(0.5*size_y/num_pts), g*(0.5*size_y/num_pts), size=y.shape)
pts_inside = np.stack((x, y), axis=-1)
for contour in contours:
    pts_inside = np.concatenate((pts_inside, contour[::12]), axis=0) #::5 pega esparsos

# outra opcao pontos internos
#all_pts = np.stack((s_x, s_y), axis=-1)
#pts_inside = all_pts[path.Path(contours[0]).contains_points(all_pts)]

# gera os triangulos e filtra os triangulos na regiao com concavidade
d = Delaunay(pts_inside)
points, triangles = d.generate_triangulation()
filtered_triangles = []
for tri in triangles:
    tri_center = np.mean(points[tri], axis=0, dtype=int)
    if bin_img[tri_center[0], tri_center[1]] == mask_value:
        filtered_triangles.append(tri)
filtered_triangles = np.array(filtered_triangles)
plt.triplot(points[:, 0], points[:, 1], filtered_triangles, "go-")
plt.show()